The below code was the first model run.  The data was processed and categorized into bins, then using one hot encoding the categorical data was converted to numeric to be used in the neural network model.  After evaluating the model, we found an accuracy of only 37.3% with a loss of 2.6.  It was easily determined changes were needed to improve the model.

In [1]:
#Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
#Import and read the dataset.
dataset_df = pd.read_csv("data/housing.csv")
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
#Identify incomplete rows
dataset_df.count()

longitude             20640
latitude              20640
housing_median_age    20640
total_rooms           20640
total_bedrooms        20433
population            20640
households            20640
median_income         20640
median_house_value    20640
ocean_proximity       20640
dtype: int64

In [4]:
#Drop all rows with missing information
dataset_df = dataset_df.dropna(how="any")
dataset_df.count()

longitude             20433
latitude              20433
housing_median_age    20433
total_rooms           20433
total_bedrooms        20433
population            20433
households            20433
median_income         20433
median_house_value    20433
ocean_proximity       20433
dtype: int64

In [5]:
#Determine the number of unique values in each column.
dataset_df.nunique()

longitude               844
latitude                861
housing_median_age       52
total_rooms            5911
total_bedrooms         1923
population             3879
households             1809
median_income         12825
median_house_value     3833
ocean_proximity           5
dtype: int64

In [6]:
#Find the minimum longitude value for binning.
dataset_df["longitude"].min()

-124.35

In [7]:
#Find the maximum longitude value for binning.
dataset_df["longitude"].max()

-114.31

In [8]:
#Bin the longitude values.

#Create bin cutoffs
longitude_bins = [-125, -124, -123, -122, -121, -120, -119, -118, -117, -116, -115, -114]

#Create bin names
longitude_names = ["-125 to -124", "-123.9 to -123", "-122.9 to -122", "-121.9 to -121", "-120.9 to -120", "-119.9 to -119", "-118.9 to -118", "-117.9 to -117", "-116.9 to -116", "-115.9 to -115", "-114.9 to -114"]

#Slice the data and place it into bins
dataset_df["longitude_category"] = pd.cut(dataset_df["longitude"], longitude_bins, labels=longitude_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122


In [9]:
#Find the minimum latitude value for binning.
dataset_df["latitude"].min()

32.54

In [10]:
#Find the maximum latitude value for binning.
dataset_df["latitude"].max()

41.95

In [11]:
#Bin the latitude values.

#Create bin cutoffs
latitude_bins = [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]

#Create bin names
latitude_names = ["32 to 33", "33.1 to 34", "34.1 to 35", "35.1 to 36", "36.1 to 37", "37.1 to 38", "38.1 to 39", "39.1 to 40", "40.1 to 41", "41.1 to 42"]

#Slice the data and place it into bins
dataset_df["latitude_category"] = pd.cut(dataset_df["latitude"], latitude_bins, labels=latitude_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38


In [12]:
#Find the minimum median_house_value for binning.
dataset_df["median_house_value"].min()

14999.0

In [13]:
#Find the maximum median_house_value for binning.
dataset_df["median_house_value"].max()

500001.0

In [14]:
#Bin the median_house_value.

#Create bin cutoffs
value_bins = [14500, 59500, 104500, 149500, 194500, 239500, 284500, 329500, 374500, 419500, 464500, 509500]

#Create bin names
value_names = ["14500-59500", "59501-104500", "104501-149500", "149501-194500", "194501-239500", "239501-284500", "284501-329500", "329501-374500", "374501-419500", "419501-464500", "464501-509500"]

#Slice the data and place it into bins
dataset_df["value_category"] = pd.cut(dataset_df["median_house_value"], value_bins, labels=value_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category,value_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500


In [15]:
#Find the maximum housing_median_age value for binning.
dataset_df["housing_median_age"].max()

52.0

In [16]:
#Bin the housing_median_age values.

#Create bin cutoffs
age_bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]

#Create bin names
age_names = ["0-5", "6-10", "11-15", "16-20", "21-25", "26-30", "31-35", "36-40", "41-45", "46-50", "51-55"]

#Slice the data and place it into bins
dataset_df["median_age_category"] = pd.cut(dataset_df["housing_median_age"], age_bins, labels=age_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category,value_category,median_age_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500,41-45
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,21-25
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55


In [17]:
#Find the maximum total_rooms value for binning.
dataset_df["total_rooms"].max()

39320.0

In [18]:
#Bin the total_rooms values.

#Create bin cutoffs
rooms_bins = [0, 3950, 7900, 11850, 15800, 19750, 23700, 27650, 31600, 35550, 39500]

#Create bin names
rooms_names = ["0-3950", "3951-7900", "7901-11850", "11851-15800", "15801-19750", "19751-23700", "23701-27650", "27651-31600", "31601-35550", "35551-39500"]

#Slice the data and place it into bins
dataset_df["total_rooms_category"] = pd.cut(dataset_df["total_rooms"], rooms_bins, labels=rooms_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category,value_category,median_age_category,total_rooms_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500,41-45,0-3950
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,21-25,3951-7900
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950


In [19]:
#Find the maximum total_bedrooms value for binning.
dataset_df["total_bedrooms"].max()

6445.0

In [20]:
#Bin the total_bedrooms values.

#Create bin cutoffs
bedrooms_bins = [0, 645, 1290, 1935, 2580, 3225, 3870, 4515, 5160, 5805, 6450]

#Create bin names
bedrooms_names = ["0-645", "646-1290", "1291-1935", "1936-2580", "2581-3225", "3226-3870", "3871-4515", "4516-5160", "5161-5805", "5806-6450"]

#Slice the data and place it into bins
dataset_df["total_bedrooms_category"] = pd.cut(dataset_df["total_bedrooms"], bedrooms_bins, labels=bedrooms_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category,value_category,median_age_category,total_rooms_category,total_bedrooms_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500,41-45,0-3950,0-645
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,21-25,3951-7900,646-1290
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645


In [21]:
#Find the maximum population value for binning.
dataset_df["population"].max()

35682.0

In [22]:
#Bin the population values.

#Create bin cutoffs
population_bins = [0, 3570, 7140, 10710, 14280, 17850, 21420, 24990, 28560, 32130, 35700]

#Create bin names
population_names = ["0-3570", "3571-7140", "7141-10710", "10711-14280", "14281-17850", "17851-21420", "21421-24990", "24991-28560", "28561-32130", "32131-35700"]

#Slice the data and place it into bins
dataset_df["population_category"] = pd.cut(dataset_df["population"], population_bins, labels=population_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category,value_category,median_age_category,total_rooms_category,total_bedrooms_category,population_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500,41-45,0-3950,0-645,0-3570
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,21-25,3951-7900,646-1290,0-3570
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570


In [23]:
#Find the maximum households value for binning.
dataset_df["households"].max()

6082.0

In [24]:
#Bin the households values.

#Create bin cutoffs
households_bins = [0, 609, 1218, 1827, 2436, 3045, 3654, 4263, 4872, 5481, 6090]

#Create bin names
households_names = ["0-609", "610-1218", "1219-1827", "1828-2436", "2437-3045", "3046-3654", "3655-4263", "4264-4872", "4873-5481", "5482-6090"]

#Slice the data and place it into bins
dataset_df["households_category"] = pd.cut(dataset_df["households"], households_bins, labels=households_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category,value_category,median_age_category,total_rooms_category,total_bedrooms_category,population_category,households_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500,41-45,0-3950,0-645,0-3570,0-609
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,21-25,3951-7900,646-1290,0-3570,610-1218
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609


In [25]:
#Find the maximum median_income value for binning.
dataset_df["median_income"].max()

15.0001

In [26]:
#Bin the median_income values.

#Create bin cutoffs
median_income_bins = [0, 1.6, 3.2, 4.8, 6.7, 8.3, 9.9, 11.5, 13.1, 14.7, 16]

#Create bin names
median_income_names = ["0-1.6", "1.6001-3.2", "3.2001-4.8", "4.8001-6.7", "6.7001-8.3", "8.3001-9.9", "9.9001-11.5", "11.5001-13.1", "13.1001-14.7", "14.7001-16"]

#Slice the data and place it into bins
dataset_df["median_income_category"] = pd.cut(dataset_df["median_income"], median_income_bins, labels=median_income_names, include_lowest=True)

#View the updated dataset
dataset_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,longitude_category,latitude_category,value_category,median_age_category,total_rooms_category,total_bedrooms_category,population_category,households_category,median_income_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500,41-45,0-3950,0-645,0-3570,0-609,8.3001-9.9
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,21-25,3951-7900,646-1290,0-3570,610-1218,8.3001-9.9
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609,6.7001-8.3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609,4.8001-6.7
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609,3.2001-4.8


In [27]:
#Drop original columns that were binned.
dataset_df = dataset_df.drop(columns=["longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income", "median_house_value"])
dataset_df.head()

,ocean_proximity,longitude_category,latitude_category,value_category,median_age_category,total_rooms_category,total_bedrooms_category,population_category,households_category,median_income_category
0,NEAR BAY,-122.9 to -122,37.1 to 38,419501-464500,41-45,0-3950,0-645,0-3570,0-609,8.3001-9.9
1,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,21-25,3951-7900,646-1290,0-3570,610-1218,8.3001-9.9
2,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609,6.7001-8.3
3,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609,4.8001-6.7
4,NEAR BAY,-122.9 to -122,37.1 to 38,329501-374500,51-55,0-3950,0-645,0-3570,0-609,3.2001-4.8


In [28]:
home_values_df = pd.DataFrame(dataset_df['value_category'])
home_values_encoded_df = pd.get_dummies(home_values_df)
home_values_encoded_df.head()

,value_category_14500-59500,value_category_59501-104500,value_category_104501-149500,value_category_149501-194500,value_category_194501-239500,value_category_239501-284500,value_category_284501-329500,value_category_329501-374500,value_category_374501-419500,value_category_419501-464500,value_category_464501-509500
0,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,True,False,False,False


In [29]:
convert_boolean = home_values_encoded_df.select_dtypes(include=['bool']).columns
home_values_encoded_df[convert_boolean] = home_values_encoded_df[convert_boolean].astype(int)

home_values_encoded_df.head()

,value_category_14500-59500,value_category_59501-104500,value_category_104501-149500,value_category_149501-194500,value_category_194501-239500,value_category_239501-284500,value_category_284501-329500,value_category_329501-374500,value_category_374501-419500,value_category_419501-464500,value_category_464501-509500
0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0


In [30]:
new_dataset_df = dataset_df.drop(columns='value_category')

new_dataset_df.head()

,ocean_proximity,longitude_category,latitude_category,median_age_category,total_rooms_category,total_bedrooms_category,population_category,households_category,median_income_category
0,NEAR BAY,-122.9 to -122,37.1 to 38,41-45,0-3950,0-645,0-3570,0-609,8.3001-9.9
1,NEAR BAY,-122.9 to -122,37.1 to 38,21-25,3951-7900,646-1290,0-3570,610-1218,8.3001-9.9
2,NEAR BAY,-122.9 to -122,37.1 to 38,51-55,0-3950,0-645,0-3570,0-609,6.7001-8.3
3,NEAR BAY,-122.9 to -122,37.1 to 38,51-55,0-3950,0-645,0-3570,0-609,4.8001-6.7
4,NEAR BAY,-122.9 to -122,37.1 to 38,51-55,0-3950,0-645,0-3570,0-609,3.2001-4.8


In [31]:
conv_dataset_df = pd.get_dummies(new_dataset_df)
conv_dataset_df.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN,longitude_category_-125 to -124,longitude_category_-123.9 to -123,longitude_category_-122.9 to -122,longitude_category_-121.9 to -121,longitude_category_-120.9 to -120,...,median_income_category_0-1.6,median_income_category_1.6001-3.2,median_income_category_3.2001-4.8,median_income_category_4.8001-6.7,median_income_category_6.7001-8.3,median_income_category_8.3001-9.9,median_income_category_9.9001-11.5,median_income_category_11.5001-13.1,median_income_category_13.1001-14.7,median_income_category_14.7001-16
0,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
3,False,False,False,True,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,True,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False


In [32]:
convert_boolean = conv_dataset_df.select_dtypes(include=['bool']).columns
conv_dataset_df[convert_boolean] = conv_dataset_df[convert_boolean].astype(int)

conv_dataset_df.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN,longitude_category_-125 to -124,longitude_category_-123.9 to -123,longitude_category_-122.9 to -122,longitude_category_-121.9 to -121,longitude_category_-120.9 to -120,...,median_income_category_0-1.6,median_income_category_1.6001-3.2,median_income_category_3.2001-4.8,median_income_category_4.8001-6.7,median_income_category_6.7001-8.3,median_income_category_8.3001-9.9,median_income_category_9.9001-11.5,median_income_category_11.5001-13.1,median_income_category_13.1001-14.7,median_income_category_14.7001-16
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [33]:
home_values_encoded_df.values

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [34]:
#Split processed data into features and target arrays
y = home_values_encoded_df.values
x = conv_dataset_df.values

#Split the processed data into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)


In [35]:
#Create a StandardScaler instance
scaler = StandardScaler()

#Fit the StandardScaler
x_scaler = scaler.fit(x_train)

#Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [36]:
nn = tf.keras.models.Sequential()

input_features = x.shape[1]

# First hidden layer
nn.add(tf.keras.layers.Dense(units=75, activation='relu',input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=11, activation='softmax'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 75)                6600      
                                                                 
 dense_1 (Dense)             (None, 50)                3800      
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 11)                561       
                                                                 
Total params: 13511 (52.78 KB)
Trainable params: 13511 (52.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
# Compile the model
nn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [38]:
# Train the model
train_model = nn.fit(x_train_scaled,y_train,epochs=500)

Epoch 1/500


479/479 [==============================] - 2s 2ms/step - loss: 1.7579 - accuracy: 0.3548
Epoch 2/500
479/479 [==============================] - 1s 2ms/step - loss: 1.5416 - accuracy: 0.4123
Epoch 3/500
479/479 [==============================] - 1s 1ms/step - loss: 1.5075 - accuracy: 0.4227
Epoch 4/500
479/479 [==============================] - 1s 1ms/step - loss: 1.4858 - accuracy: 0.4297
Epoch 5/500
479/479 [==============================] - 1s 1ms/step - loss: 1.4724 - accuracy: 0.4335
Epoch 6/500
479/479 [==============================] - 1s 1ms/step - loss: 1.4608 - accuracy: 0.4373
Epoch 7/500
479/479 [==============================] - 1s 2ms/step - loss: 1.4515 - accuracy: 0.4407
Epoch 8/500
479/479 [==============================] - 1s 2ms/step - loss: 1.4411 - accuracy: 0.4407
Epoch 9/500
479/479 [==============================] - 1s 1ms/step - loss: 1.4331 - accuracy: 0.4466
Epoch 10/500
479/479 [==============================] - 1s 1ms/step - loss: 1.4225 - accuracy: 0.4496
E

In [39]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

160/160 - 1s - loss: 2.6557 - accuracy: 0.3731 - 681ms/epoch - 4ms/step
Loss: 2.655733346939087, Accuracy: 0.37306714057922363
